In [ ]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sys.path.append('script')
from script import dbconn
pgconn = dbconn.db_connection_psycopg()

In [ ]:
# Fectching data from the postgreSql database and put the value on raw_df
raw_df = dbconn.db_read_table_psycopg(pgconn,'xdr_data')

In [ ]:
raw_df.info

In [ ]:
raw_df.describe

In [ ]:
raw_df.columns

In [ ]:
#the top 10 handsets used by the customers
top_10_handsets = raw_df['Handset Type'].value_counts().head(10)
print(top_10_handsets)

In [ ]:
#the top 3 handset manufacturers
top_3_manufacturers = raw_df['Handset Manufacturer'].value_counts().head(3)
print(top_3_manufacturers)

In [ ]:
# the top 5 handsets per top 3 handset manufacturer
top_3_manufacturers = raw_df['Handset Manufacturer'].value_counts().head(3).index

for manufacturer in top_3_manufacturers:
    top_5_handsets = raw_df.loc[raw_df['Handset Manufacturer'] == manufacturer, 'Handset Type'].value_counts().head(5)
    print(f"Top 5 handsets for {manufacturer}:")
    print(top_5_handsets)
    print()

                                   TASK 2.1

In [ ]:
#number of xDR sessions
user_sessions = raw_df.groupby('MSISDN/Number')['Bearer Id'].count().reset_index()
user_sessions.columns = ['MSISDN/Number', 'Number of xDR Sessions']
print(user_sessions)

In [ ]:
#Session duration
user_session_duration = raw_df.groupby('MSISDN/Number')['Dur. (ms)'].sum().reset_index()
user_session_duration.columns = ['MSISDN/Number', 'Session Duration']
print(user_session_duration)

In [ ]:
#the total download (DL) and upload (UL) data
user_data = raw_df.groupby('MSISDN/Number').agg({
    'Total DL (Bytes)': 'sum',
    'Total UL (Bytes)': 'sum'
}).reset_index()
user_data.columns = ['MSISDN/Number', 'Total DL Data', 'Total UL Data']
print(user_data)

In [ ]:
#the total data volume (in Bytes) 
user_session_data = raw_df.groupby('MSISDN/Number').agg({
    'Total UL (Bytes)': 'sum',
    'Total DL (Bytes)': 'sum'
}).reset_index()
user_session_data['Total Data Volume'] = user_session_data['Total UL (Bytes)'] + user_session_data['Total DL (Bytes)']
user_session_data = user_session_data[['MSISDN/Number', 'Total Data Volume']]
print(user_session_data)

                                     TASK 2.2

In [ ]:
raw_df.info()

In [ ]:
raw_df.isna().sum()

In [ ]:
raw_df.describe

In [ ]:
#percent of missing data

def percent_missing(df):
    # Calculate total number of cells in dataframe
    totalCells = np.product(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    percentageMissing = (totalMissing / totalCells) * 100

    print("The dataset contains", round(percentageMissing, 2), "%", "missing values.")

percent_missing(raw_df)

In [ ]:
#Identify and replace outliers and missing values with column mean



# Replace missing values with column mean
raw_df.fillna(raw_df.mean(), inplace=True)

# Identify and replace outliers with column mean
num_columns = raw_df.select_dtypes(include=[np.number]).columns

for col in num_columns:
    z_scores = (raw_df[col] - raw_df[col].mean()) / raw_df[col].std()
    outliers = (z_scores > 3) | (z_scores < -3)
    raw_df[col][outliers] = raw_df[col].mean()

# Verify missing values and outliers have been treated
missing_values_after_treatment = raw_df.isnull().sum()
print("Missing Values After Treatment:\n", missing_values_after_treatment)

In [ ]:
# Calculate the percentage of missing values in each column
missing_percent = (raw_df.isnull().sum() / len(raw_df)) * 100

# Drop columns with more than 30% missing values
columns_to_drop = missing_percent[missing_percent > 30].index
df_clean = raw_df.drop(columns_to_drop, axis=1)

# Print the shape of the cleaned DataFrame
print("Shape of cleaned DataFrame:", df_clean.shape)

In [ ]:
missing_values = raw_df.isna().sum()
print(missing_values)

In [ ]:
#Solving The rest of missing values
def fix_missing_ffill(df, col):
    df[col] = df[col].fillna(method='ffill')
    return df[col]

raw_df['Start'] = fix_missing_ffill(raw_df, 'Start')
raw_df['End'] = fix_missing_ffill(raw_df, 'End')
raw_df['Last Location Name'] = fix_missing_ffill(raw_df, 'Last Location Name')

missing_values = raw_df.isna().sum()
print(missing_values)

In [ ]:

# Calculate basic metrics
metrics = raw_df.describe()
mean = metrics.loc['mean']
median = metrics.loc['50%']
mode = raw_df.mode().iloc[0]
minimum = metrics.loc['min']
maximum = metrics.loc['max']
std_deviation = metrics.loc['std']

# Print the basic metrics
print("Mean:\n", mean)
print("\nMedian:\n", median)
print("\nMode:\n", mode)
print("\nMinimum:\n", minimum)
print("\nMaximum:\n", maximum)
print("\nStandard Deviation:\n", std_deviation)


In [ ]:
#a Non-Graphical Univariate Analysis by computing dispersion parameters for each quantitative variable

# Select quantitative variables in the dataset
quantitative_vars = raw_df.select_dtypes(include=[np.number])

# Compute dispersion parameters for each quantitative variable
dispersion_parameters = quantitative_vars.agg(['mean', 'median', 'std', 'min', 'max', 'var'])

# Print the dispersion parameters
print("Dispersion Parameters:\n", dispersion_parameters)

In [ ]:
#a Graphical Univariate Analysis for each variable

# Select variables in the dataset
variables = raw_df.columns

# Plotting options for each variable
for variable in variables:
    if raw_df[variable].dtype == 'int64' or raw_df[variable].dtype == 'float64':
        # For numeric variables (continuous or discrete)
        plt.figure(figsize=(8, 6))
        sns.histplot(data=raw_df, x=variable, kde=True)
        plt.title(f'Distribution of {variable}')
        plt.xlabel(variable)
        plt.ylabel('Frequency')
        plt.show()
    else:
        # For categorical variables
        plt.figure(figsize=(8, 6))
        sns.countplot(data=raw_df, x=variable)
        plt.title(f'Count of {variable}')
        plt.xlabel(variable)
        plt.ylabel('Count')
        plt.xticks(rotation=90)
        plt.show()

In [ ]:
#Correlation Analysis
variables = ['Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)',
             'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)',
             'Other DL (Bytes)']

# Subset the DataFrame with the selected variables
subset_df = raw_df[variables]

# Compute the correlation matrix
correlation_matrix = subset_df.corr()

# Print the correlation matrix
print("Correlation Matrix:")
print(correlation_matrix)

In [ ]:
#Dimensionality Reduction 

# Assuming you have a DataFrame 'raw_df' with the relevant variables
variables = ['Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)',
             'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)',
             'Other DL (Bytes)']

# Subset the DataFrame with the selected variables
subset_df = raw_df[variables]

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(subset_df)

# Perform PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_data)

# Create a DataFrame with the principal components
pc_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# Explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Interpretation of the results
print("Interpretation of PCA results:")
print("- The first principal component (PC1) explains", round(explained_variance_ratio[0] * 100, 2), "% of the variance in the data.")
print("- The second principal component (PC2) explains", round(explained_variance_ratio[1] * 100, 2), "% of the variance in the data.")
print("- PC1 captures the most significant patterns and trends in the data, such as overall data usage level.")
print("- PC2 captures additional variation that is orthogonal to PC1 and represents specific usage patterns or differences between the applications.")